In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
sample_submission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [3]:
#preprocessing ExterQual and ExterCond into one feature
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data = [train_data,test_data]
for dataset in data:
    dataset['ExterQual'] = label_encoder.fit_transform(dataset['ExterQual'])
    dataset['ExterCond'] = label_encoder.fit_transform(dataset['ExterCond'])
    dataset['ExterQualCond'] = dataset['ExterQual']*dataset['ExterCond']

In [4]:
#preprocessing MasVnrType and MasVnrArea into one feature
from sklearn.preprocessing import LabelEncoder
mean_value = train_data['MasVnrArea'].mean()
label_encoder = LabelEncoder()
common = 'BrkFace'
data = [train_data,test_data]
for dataset in data:
    dataset['MasVnrType'] = dataset['MasVnrType'].fillna(common)
    dataset['MasVnrType'] = label_encoder.fit_transform(dataset['MasVnrType'])
    dataset['MasVnrArea'] = dataset['MasVnrArea'].fillna(mean_value)
    dataset['MasVnrArea'] = dataset['MasVnrArea']*dataset['MasVnrType']
    dataset['MasVnrArea'] = dataset['MasVnrArea'].astype(int)
    

In [5]:
#custom labelling of RoofStyle
values  = {'Gambrel':1,'Gable':2,'Mansard':3,'Flat':4,'Hip':5,'Shed':6}
data = [train_data,test_data]
for dataset in data:
    dataset['RoofStyle'] = dataset['RoofStyle'].map(values)

In [6]:
#making a feature out off YearBuilt and YearRemodAdd
data = [train_data,test_data]
for datasets in data:
    datasets['YearDifference'] = (datasets['YearRemodAdd'] - datasets['YearBuilt']).astype(int)

In [7]:
#categorising the YearDifference feature
data = [train_data,test_data]
for dataset in data:
    dataset.loc[dataset['YearDifference']<=60 ,'YearDifference'] = 1
    dataset.loc[(dataset['YearDifference']>60) & (dataset['YearDifference']<=100),'YearDifference'] = 0
    dataset.loc[dataset['YearDifference']>100,'YearDifference'] = 2
    
    

In [8]:
#label encoding feature BldgType,HouseStyle
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data= [train_data,test_data]
for dataset in data:
    dataset['BldgType'] = label_encoder.fit_transform(dataset['BldgType'])
    dataset['HouseStyle'] = label_encoder.fit_transform(dataset['HouseStyle'])
    

In [9]:
#mapping LandSlope into integer values
values = {'Sev':3,'Mod':2,'Gtl':1}
data = [train_data,test_data]
for dataset in data:
    dataset['LandSlope'] = dataset['LandSlope'].map(values)
    

In [10]:
#mapping LotConfig to integer values #preprocessing
values = {'Inside':1,'Corner':1,'CulDSac':3,'FR2':1,'FR3':3}
data = [train_data,test_data]
for datasets in data:
    datasets['LotConfig'] = datasets['LotConfig'].map(values)


In [11]:
#preprocessing LotFrontage feature and filling nan values with the mean
data = [train_data,test_data]
mean_value = train_data['LotFrontage'].mean()
for datasets in data:
    datasets['LotFrontage'] = datasets['LotFrontage'].fillna(mean_value)
    datasets['LotFrontage'] = datasets['LotFrontage'].astype(int)
    

    

In [12]:

features = ['MSZoning','LotShape','Neighborhood']
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

X['MSSubClass'] = train_data['MSSubClass']
X['LotFrontage'] = train_data['LotFrontage']
X['LotArea'] = train_data['LotArea']
X['LotConfig'] = train_data['LotConfig']
X['LandSlope'] = train_data['LandSlope']
X['BldgType'] = train_data['BldgType']
X['HouseStyle'] = train_data['HouseStyle']
X['OverallQual'] = train_data['OverallQual']
X['OverallCond'] = train_data['OverallCond']
X['YearDifference'] = train_data['YearDifference']
X['RoofStyle'] = train_data['RoofStyle']
X['MasVnrArea'] = train_data['MasVnrArea']
X['ExterQualCond'] = train_data['ExterQualCond']









X_test['MSSubClass'] = test_data['MSSubClass']
X_test['LotFrontage'] = test_data['LotFrontage']
X_test['LotArea'] = test_data['LotArea']
X_test['LotConfig'] = test_data['LotConfig']
X_test['LandSlope'] = test_data['LandSlope']
X_test['BldgType'] = test_data['BldgType']
X_test['HouseStyle'] = test_data['HouseStyle']
X_test['OverallQual'] = test_data['OverallQual']
X_test['OverallCond'] = test_data['OverallCond']
X_test['YearDifference'] = test_data['YearDifference']
X_test['RoofStyle'] = test_data['RoofStyle']
X_test['MasVnrArea'] = test_data['MasVnrArea']
X_test['ExterQualCond'] = test_data['ExterQualCond']
y = train_data['SalePrice']

In [13]:
X.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,Neighborhood_Blmngtn,...,LotConfig,LandSlope,BldgType,HouseStyle,OverallQual,OverallCond,YearDifference,RoofStyle,MasVnrArea,ExterQualCond
0,0,0,0,1,0,0,0,0,1,0,...,1,1,0,5,7,5,1,2,196,8
1,0,0,0,1,0,0,0,0,1,0,...,1,1,0,2,6,8,1,2,0,12
2,0,0,0,1,0,1,0,0,0,0,...,1,1,0,5,7,5,1,2,162,8
3,0,0,0,1,0,1,0,0,0,0,...,1,1,0,5,7,5,1,2,0,12
4,0,0,0,1,0,1,0,0,0,0,...,1,1,0,5,8,5,1,2,350,8


In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

X_train,X_testing,y_train,y_testing = train_test_split(X,y,random_state = 1)

model = LinearRegression()
model.fit(X_train,y_train)
predictions = model.predict(X_testing)
mean_absolute_error(y_testing,predictions)

#output = pd.DataFrame({'Id': test_data.Id, 'SalePrice': predictions})
#output.to_csv('my_submission3.csv', index=False)


25477.90075242058